In [ ]:
'''
pip install -U FlagEmbedding
pip install -U langchain-community
pip install -v gptqmodel --no-build-isolation
pip install --upgrade optimum transformers
pip install faiss-cpu
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing Packages and Models

In [1]:
from langchain.vectorstores import FAISS
from FlagEmbedding import FlagAutoModel

embed_model = FlagAutoModel.from_finetuned('BAAI/bge-base-en-v1.5',
                                      query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                                      use_fp16=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "thesven/Mistral-7B-Instruct-v0.3-GPTQ",
    device_map = 'auto'
)
tokenizer = AutoTokenizer.from_pretrained("thesven/Mistral-7B-Instruct-v0.3-GPTQ")


INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.      
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                              
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`                               


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at thesven/Mistral-7B-Instruct-v0.3-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_pro

INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                    
INFO  Format: Converting GPTQ v1 to v2                                                             
INFO  Format: Conversion complete: 0.07536005973815918s                                            


In [18]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
vector_store_path = "/content/drive/MyDrive/my_vector_store"
vector_store = FAISS.load_local(vector_store_path, embeddings=embed_model, allow_dangerous_deserialization=True)

Retrieve

In [64]:
query = "Can you take HYDROCHLOROTHIAZIDE if you are pregnant?"
embed_query = embed_model.encode([query])

In [65]:
if "pregnant" in query.lower():
    category = 'specific population usage'

elif "uses" in query.lower():
    category = 'general'

else:
    category = 'general'

In [66]:
results = vector_store.similarity_search_by_vector(
    embed_query[0],
    k=3,
    filter={"category": category}
)

cleaned_context = [doc.page_content for doc in results]
cleaned_context

['Name: HYDROCHLOROTHIAZIDE | Use in Specific Populations: Use in Pregnancy: Routine use of diuretics during normal pregnancy is inappropriate and exposes mother and fetus to unnecessary hazard. Diuretics do not prevent development of toxemia of pregnancy and there is no satisfactory evidence that they are useful in the treatment of toxemia. Edema during pregnancy may arise from pathologic causes or from the physiologic and mechanical consequences of pregnancy. Thiazides are indicated in pregnancy when edema is due to pathologic causes, just as they are in the absence of pregnancy (see PRECAUTIONS: Pregnancy ). Dependent edema in pregnancy, resulting from restriction of venous return by the gravid uterus, is properly treated through elevation of the lower extremities and use of support stockings. Use of diuretics to lower intravascular volume in this instance is illogical and unnecessary. During normal pregnancy there is hypervolemia which is not harmful to the fetus or the mother in t

Augmentation

In [67]:
prompt_template = """
You are an expert medical assistant explaining medicine to someone without any prior knowledge.
Use the provided context to answer the user's question accurately.

Context: {context}

Question: {question}

Answer:
"""

In [68]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
mistral_llm = HuggingFacePipeline(
    pipeline=pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
)

Device set to use cuda:0


In [69]:
qa_chain = prompt | mistral_llm

Generate

In [70]:
response = qa_chain.invoke({'context' : cleaned_context, 'question' : query})

response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\nYou are an expert medical assistant explaining medicine to someone without any prior knowledge. \nUse the provided context to answer the user's question accurately.\n\nContext: ['Name: HYDROCHLOROTHIAZIDE | Use in Specific Populations: Use in Pregnancy: Routine use of diuretics during normal pregnancy is inappropriate and exposes mother and fetus to unnecessary hazard. Diuretics do not prevent development of toxemia of pregnancy and there is no satisfactory evidence that they are useful in the treatment of toxemia. Edema during pregnancy may arise from pathologic causes or from the physiologic and mechanical consequences of pregnancy. Thiazides are indicated in pregnancy when edema is due to pathologic causes, just as they are in the absence of pregnancy (see PRECAUTIONS: Pregnancy ). Dependent edema in pregnancy, resulting from restriction of venous return by the gravid uterus, is properly treated through elevation of the lower extremities and use of support stockings. Use of diure

In [71]:
def extract_answer(text):
    # Ensure "Answer: " exists in the string
    if "Answer:" in text:
        return text.split("Answer:", 1)[1]

In [72]:
result = extract_answer(response)
result

'\nRoutine use of HYDuring pregnancy is not recommended during pregnancy. The use of HYDROCHLOROTHZINE is not recommended for routine use during pregnancy. It is not recommended during pregnancy. Pregnancy. HYD is not recommended for pregnant women. It is not recommended to use of HYDuring pregnancy. It is not recommended for routine use of HYDuring pregnancy.\n\n\n\n\n\nIn pregnancy, Hydrochlorithiazide is not recommended for pregnant women who are not recommended during pregnancy.\n\n\n\n\n\n\nThe use of Hydrochlorothiazide.\n\n\n\n\n\nThe use of HYDuring pregnancy.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe use of HYD\n\n\n\nThe use of Hydrochloro\n\nThe use of HY\n\n\n\nThe use of HydrochlorotHydrochlorothiazide\n\nThe\n\n\n\nThe use of Hydrochloro\n\n\nThe use of Hydrochlorin\n\n\n\n\n\nThe\n\n\nThe\n\n\n\n\n\n\n\nthe\n\n\n\n\n\n\n\n\n\nThe\n\n\n\n\nThe\nThe\nThe\nof\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

Testing cosine similarity of the context

In [73]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

embedded_context = embed_model.encode(cleaned_context)
similarity_scores = cosine_similarity(embed_query, embedded_context)

for score in similarity_scores:
  print(score)


[0.80863949 0.77278659 0.7724666 ]


In [ ]:
# Insert text below to test cosine similarity
text = "BENZOCAINE,Summary of BENZOCAINE | Uses: Use for the temporary relief of pain due to toothaches * to help protect against infection of minor oral irritation and  |  and  | Reactions: ,,,Name: BENZOCAINE | Dosage and Administration: Directions cut open tip of tube on score mark * adults and children 2 years of age and older: apply a small amount of product to the cavity and around the gum surrounding the teeth. Use up to 4 times daily or as directed by a dentist or doctor * children under 12 years of age should be supervised in the use of this product * children under 2 years of age: do not use"

In [59]:
embed_text = embed_model.encode(text)
embed_text = embed_text.reshape(1, -1)
embed_query = embed_query.reshape(1, -1)

similarity_scores = cosine_similarity(embed_text, embed_query)

print(similarity_scores)

[[0.64219971]]
